# This code renders the parameter config files for CFE-Cver to CFE-Python version

Written by Ryoko Araki (San Diego State University & UCSB, raraki8159@sdsu.edu) in 2023 SI 

In [49]:
# %%
import pandas as pd
import numpy as np
import os
import sys
import json
import re 

from omegaconf import DictConfig, OmegaConf
import hydra
import yaml
import warnings
from tqdm import tqdm


## Read the config

In [34]:
with open('config.yaml', 'r') as f:
    cfg = yaml.safe_load(f)
    
# define GIUH and soil params files
GIUH_soil_dir = cfg['io_dir']['cfe_c_config_dir'].replace("${cwd}", "..")

# define basin list dir
basin_dir = cfg['io_dir']['gauch_2020_dir'].replace("${cwd}", "..")
basin_filename = cfg['model_settings']['basin_file'].replace("${cwd}", "..") # It was 516 basin in 2022 code 

# define camel dataset dir
camels_attr_dir = cfg['io_dir']['ucar_dir'].replace("${cwd}", "..")

# define atmospheric forcing file dir
forcing_path = cfg['io_dir']['nldas_forcing_dir'].replace("${cwd}", "..")

# define dir for exported json
config_dir = cfg['io_dir']['config_dir'].replace("${cwd}", "..")

soil_scheme = cfg['CFE_config']['soil_scheme']


In [35]:
with open(basin_filename, "r") as f:
    basin_list = pd.read_csv(f, header=None)

with open(basin_filename, 'r') as file:
    lines = file.readlines()
    # Remove leading/trailing whitespaces and newline characters
    lines = [line.strip() for line in lines]
basin_list_str = lines

In [40]:
basin_attributes = 


## Count the number of parameter files in the C version

In [55]:
file_count = sum(1 for _ in os.scandir(GIUH_soil_dir) if _.is_file())

pattern = r'(\d+)\_bmi_config_cfe_pass.txt'
gauge_ids_in_c_config = []
for filename in os.listdir(GIUH_soil_dir):
    match = re.match(pattern, filename)
    if match:
        gauge_id = match.group(1)
        gauge_ids_in_c_config.append(gauge_id)
        
print(f"Number of config files that 2022 team received from Luciana: {file_count}")
common_gauges = set(gauge_ids_in_c_config) & set(basin_list_str)
print(f"Number of the gauges matching with our basin of interest: {len(common_gauges)}/516")

if len(common_gauges) != 516:
    warnings.warn("Config missing in either of the folders")
else:
    None


Number of config files that 2022 team received from Luciana: 519
Number of the gauges matching with our basin of interest: 516/516


## Read and render the config files (C -> BMIpy readable)

In [63]:

def load_config(GIUH_soil_dir, gauge_id):
    # get giuh and soil param file
    giuh_soil_file = os.path.join(GIUH_soil_dir, f'{gauge_id}_bmi_config_cfe_pass.txt')
    with open(giuh_soil_file, "r") as f:
        text = f.read()
    return text

def render_config(gauge_id, text, default_values):
    
    # Initialize
    parameters = {}
    parameters['soil_params'] = {}
    lines = text.strip().split("\n")
    
    parameters['forcing_file'] = os.path.join(forcing_path, f'{gauge_id}_hourly_nldas.csv')
    parameters['catchment_area_km2'] = 

    # Loop through the lines 
    for line in lines:
        key, value = line.split("=")
        key_parts = key.split(".")
        param_name = key_parts[-1].strip()
        
        if "[" in value:
            value = value.split("[")[0].strip()
        
        if param_name in default_values and value != "NaN":
            
            if "," in value:
                value = [float(v) for v in value.split(",")]
            else:
                try:
                    value = float(value)
                except ValueError:
                    value = default_values[param_name]
                    Warning('Detected NaN in original config file. Replaced with default values.')
            
            if param_name == "b":
                param_name = "bb"

            if param_name in ["depth", "satdk", "satpsi", "slop", "smcmax", "wltsmc", "bb"]:
                parameters["soil_params"][param_name] = value
            else:        
                parameters[param_name] = value

    # Some default CFE-py parameters not generated through Hydrofabric
    parameters['stand_alone'] = 1 
    parameters['unit_test'] = 0
    parameters['compare_results_file'] = ""
    parameters['partition_scheme'] = "Schaake"
    parameters['soil_scheme'] = soil_scheme
    parameters['soil_params']["D"] = 2.0
    parameters["soil_params"]["mult"] = 1000.0

    return parameters

def save_rendered_config(gauge_id, config_dir, parameters):
    # save and export json files
    json_filename = f'cat_{gauge_id}_bmi_config_cfe___ryokotesting.json'
    json_file = os.path.join(config_dir, json_filename)

    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(parameters, f, ensure_ascii=False, indent=4, separators=(',', ':'))

In [64]:
default_values = {
    "depth": 2.0,
    "b": 5.651162790697675,
    "satdk": 1.28959488372093e-05,
    "satpsi": 0.0662031693860465,
    "slop": 0.013278359525581396,
    "smcmax": 0.46032193756744183,
    "wltsmc": 0.023237335009302328,
    "expon": 1.0,
    "expon_secondary": 1.0,
    "refkdt": 1.0, 
    "max_gw_storage": 0.011877697944999994,
    "Cgw": 1.8e-05,
    "gw_storage": 0.05,
    "alpha_fc":0.33,
    "K_nash": 0.03,
    "K_lf": 0.01,
    "nash_storage": "0.0,0.0",
    "giuh_ordinates": "0.1, 0.2, 0.4, 0.2, 0.1"
}

gauge_id = basin_list_str[0]
# for gauge_id in tqdm(basin_list_str):
for gauge_id in (basin_list_str[0]):
    text = load_config(GIUH_soil_dir, gauge_id)
    parameters = render_config(gauge_id, text, default_values)
    save_rendered_config(gauge_id, config_dir, parameters)


## Check if all the configs are generated

In [ ]:
file_count = sum(1 for _ in os.scandir(GIUH_soil_dir) if _.is_file())

pattern = r'(\d+)\_bmi_config_cfe_pass.txt'
gauge_ids_in_c_config = []
for filename in os.listdir(GIUH_soil_dir):
    match = re.match(pattern, filename)
    if match:
        gauge_id = match.group(1)
        gauge_ids_in_c_config.append(gauge_id)
        
print(f"Number of config files that 2022 team received from Luciana: {file_count}")
common_gauges = set(gauge_ids_in_c_config) & set(basin_list_str)
print(f"Number of the gauges matching with our basin of interest: {len(common_gauges)}/516")

if len(common_gauges) != 516:
    warnings.warn("Config missing in either of the folders")
else:
    None
